Тестирование проводится на NVIDIA GeForce RTX 4070 Laptop GPU 8GB

<a name='1-1'></a>
## Тестирование Обученного Quartznet15x5 
Размер 74 MB

2050 примеров:

62.7 секунды

0.03 секунды на 1 пример

WER 27.51%

In [1]:
import nemo.collections.asr as nemo_asr
from nemo.core.classes import ModelPT
import json
import os
import numpy as np
import torch
from ruamel.yaml import YAML
import torchaudio
import time
from omegaconf import DictConfig, OmegaConf, open_dict

c:\Users\vdovichev\anaconda3\envs\nemo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[NeMo W 2024-06-12 16:14:14 transformer_bpe_models:59] Could not import NeMo NLP collection which is required for speech translation model.


In [24]:
# загружаем модель NeMo
asr_model = ModelPT.restore_from(r"C:\Users\vdovichev\Documents\VKR\models_and_configs\bestQuartz1206.nemo", map_location = 'cuda')

[NeMo W 2024-06-12 16:30:54 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - /home/jupyter/datasphere/datasets/RuDevices/RuDevicesDataset/manifestRuDevices.jsonl
    - /home/jupyter/datasphere/datasets/STTandCustomer/OpenSTTruDatasets/asr_calls_2_val/manifestSTTcalls.jsonl
    - /home/jupyter/datasphere/datasets/STTandCustomer/OpenSTTruDatasets/buriy_audiobooks_2_val/manifestSTTaudiobooks.jsonl
    - /home/jupyter/datasphere/datasets/STTandCustomer/OpenSTTruDatasets/public_youtube700_val/manifestSTTyoutube.jsonl
    sample_rate: 16000
    labels:
    - ' '
    - а
    - б
    - в
    - г
    - д
    - е
    - ж
    - з
    - и
    - й
    - к
    - л
    - м
    - н
    - о
    - п
    - р
    - с
    - т
    - у
    - ф
    - х
    - ц
    - ч
    - ш
    - щ
    - ъ
    - ы
    - ь
    - э
    - ю
    - я

[NeMo I 2024-06-12 16:30:54 features:289] PADDING: 16
[NeMo I 2024-06-12 16:30:54 save_restore_connector:249] Model EncDecCTCModel was successfully restored from C:\Users\vdovichev\Documents\VKR\models_and_configs\bestQuartz1206.nemo.


In [25]:
print(asr_model.device) # проверим что cuda подключилась

cuda:0


In [26]:
yaml = YAML(typ='safe')
with open(r"C:\Users\vdovichev\Documents\VKR\models_and_configs\bestQuartz1206_config.yaml") as f:
        config_model = yaml.load(f)
preprocessor = nemo_asr.models.EncDecCTCModel.from_config_dict(config_model['preprocessor'])
decoder = nemo_asr.metrics.wer.CTCDecoding(config_model['decoding'], vocabulary=config_model['decoder']['vocabulary'])

[NeMo I 2024-06-12 16:30:59 features:289] PADDING: 16


In [27]:
manifest_paths = [
    r"C:\Users\vdovichev\Documents\proj1-STT\DatasetForFineTune\RuDevicesDataset\manifestValidationRuDevices.jsonl",
    r"C:\Users\vdovichev\Documents\proj1-STT\DatasetForFineTune\OpenSTTruDatasets\asr_calls_2_val\manifestValidationSTTcalls.jsonl",
    r"C:\Users\vdovichev\Documents\proj1-STT\DatasetForFineTune\OpenSTTruDatasets\buriy_audiobooks_2_val\manifestValidationSTTaudiobooks.jsonl",
    r"C:\Users\vdovichev\Documents\proj1-STT\DatasetForFineTune\OpenSTTruDatasets\public_youtube700_val\manifestValidationSTTyoutube.jsonl"
]

base_dirs = [
    r"C:\Users\vdovichev\Documents\proj1-STT\DatasetForFineTune\RuDevicesDataset",
    r"C:\Users\vdovichev\Documents\proj1-STT\DatasetForFineTune\OpenSTTruDatasets\asr_calls_2_val",
    r"C:\Users\vdovichev\Documents\proj1-STT\DatasetForFineTune\OpenSTTruDatasets\buriy_audiobooks_2_val",
    r"C:\Users\vdovichev\Documents\proj1-STT\DatasetForFineTune\OpenSTTruDatasets\public_youtube700_val",
]

In [28]:
asr_model.eval()
asr_model.encoder.freeze()
asr_model.decoder.freeze()

num_of_audio_all = 0

hypotheses = []
y_target = []

total_time = 0
start_time = time.time()

for manifest_path, base_dir in zip(manifest_paths, base_dirs):
    print(manifest_path)
    with open(manifest_path, 'r', encoding='utf-8') as f:
        manifest_data = [json.loads(line) for line in f]
    
    num_of_audio = len(manifest_data)
    num_of_audio_all += num_of_audio

    for audio_index in range(num_of_audio):
        current_audio_entry = manifest_data[audio_index]
        y_target.append(current_audio_entry.get('text', ''))

        current_audio_filename = current_audio_entry.get('audio_filepath', '')
        file_path = os.path.join(base_dir, current_audio_filename)
        #print(file_path)

        waveform, sample_rate = torchaudio.load(file_path)
        processed_signal, processed_signal_len = preprocessor(input_signal=waveform, length=torch.tensor([len(waveform[0])]))

        processed_signal = processed_signal.to('cuda')
        processed_signal_len = processed_signal_len.to('cuda')

        encoder_output = asr_model.encoder(audio_signal=processed_signal, length=processed_signal_len)
        logits = asr_model.decoder(encoder_output=encoder_output[0])
        current_hypotheses, _ = decoder.ctc_decoder_predictions_tensor(logits, decoder_lengths=processed_signal_len)
        hypotheses.extend(current_hypotheses)

total_time = time.time() - start_time

C:\Users\vdovichev\Documents\proj1-STT\DatasetForFineTune\RuDevicesDataset\manifestValidationRuDevices.jsonl
C:\Users\vdovichev\Documents\proj1-STT\DatasetForFineTune\OpenSTTruDatasets\asr_calls_2_val\manifestValidationSTTcalls.jsonl
C:\Users\vdovichev\Documents\proj1-STT\DatasetForFineTune\OpenSTTruDatasets\buriy_audiobooks_2_val\manifestValidationSTTaudiobooks.jsonl
C:\Users\vdovichev\Documents\proj1-STT\DatasetForFineTune\OpenSTTruDatasets\public_youtube700_val\manifestValidationSTTyoutube.jsonl


In [29]:
print(f"Число аудио: {num_of_audio_all}")
print(f"Общее время: {total_time:.2f} секунд")
print(f"Среднее время на одно аудио: {total_time/num_of_audio_all:.2f} секунд")

Число аудио: 2050
Общее время: 62.71 секунд
Среднее время на одно аудио: 0.03 секунд


In [30]:
# from jiwer import wer не подходит, т.к. он не работает, если в гипотезе есть пустые ("") аудио
print(f"Word Error Rate (WER): {nemo_asr.metrics.wer.word_error_rate(hypotheses, y_target) * 100:.2f}%")

Word Error Rate (WER): 27.51%
